In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
sns.set()
import matplotlib.pyplot as plt 

In [ ]:
df_o2 = pd.read_csv('/kaggle/input/heart-attack-analysis-prediction-dataset/o2Saturation.csv')
df_o2.head()

In [ ]:
plt.plot(df_o2.iloc[:, 0].values, 'o')
plt.xlabel('id')
plt.ylabel('SPO2')

In [ ]:
df = pd.read_csv('/kaggle/input/heart-attack-analysis-prediction-dataset/heart.csv')
df.head()

In [ ]:
df.info()

In [ ]:
sns.countplot(df['output'])
plt.show()

In [ ]:
df.describe()

# let us see the correlation table 

In [ ]:
plt.figure(figsize=(32, 16))
heatmap = sns.heatmap(df.corr(), vmin=-1, vmax=1, annot=True)

heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':12}, pad=12);

# select x: input features

# and y: output 

In [ ]:
x = df.drop(columns= ['output' ]).values
y = df['output'].values

In [ ]:
# ANOVA feature selection for numeric input and categorical output
from sklearn.datasets import make_classification
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
# define feature selection: 8 from the correlation table
fs = SelectKBest(score_func=f_classif, k=8)
# apply feature selection
X_selected = fs.fit_transform(x, y)
print(X_selected.shape)
from sklearn import svm
from sklearn.model_selection import train_test_split

# devide the dataset into training and testing dataset

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.33, random_state=42)

# calculate the class weights

In [ ]:
# class weight 
(len(y_train) / (2 * np.bincount(y_train)))

# initiate the model

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [ ]:

clf1 = RandomForestClassifier(n_estimators=100, max_depth=30, random_state=0, min_samples_split=4,class_weight={0:1.05729167, 1:0.94859813} )
clf1.fit(X_train, y_train)

# for testing 
y_predicted1 = clf1.predict(X_test)
print('testing accuracy: ', accuracy_score(y_test, y_predicted1))

#
print('confusion martrix: ')




plt.figure(figsize=(14,4))
plt.subplot(121)
heatmap = sns.heatmap(confusion_matrix(y_test, y_predicted1), annot=True)

heatmap.set_title('confusion  matrix for testing', fontdict={'fontsize':12}, pad=12);




# for training dataset

y_predicted1 = clf1.predict(X_train)
print('training accuracy: ', accuracy_score(y_train, y_predicted1))

#
print('confusion martrix: ')

#plt.figure(figsize=(4,4))
plt.subplot(122)
heatmap = sns.heatmap(confusion_matrix(y_train, y_predicted1), annot=True)

heatmap.set_title('confusion  matrix for training', fontdict={'fontsize':12}, pad=12);

# Fine tune the model further: Randomized search

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from pprint import pprint

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 20, stop = 200, num = 5)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(1, 45, num = 3)]
# Minimum number of samples required to split a node
min_samples_split = [5, 10]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split}

pprint(random_grid)

In [ ]:
# Use the random grid to search for best hyperparameters

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = clf1, param_distributions = random_grid, n_iter = 10, cv = 10, verbose=2, random_state=42, n_jobs = -1, scoring='neg_mean_squared_error')
# Fit the random search model
rf_random.fit(X_train, y_train)

In [ ]:
#now let's how the RMSE changes for each parameter configuration
cvres2 = rf_random.cv_results_
for mean_score, params in zip(cvres2["mean_test_score"], cvres2["params"]):
    print(np.sqrt(-mean_score), params)

In [ ]:
rf_random.best_params_

In [ ]:
rf_random.best_estimator_

In [ ]:


# for testing 
y_predicted1 = rf_random.best_estimator_.predict(X_test)
print('testing accuracy: ', accuracy_score(y_test, y_predicted1))

#
print('confusion martrix: ')




plt.figure(figsize=(14,4))
plt.subplot(121)
heatmap = sns.heatmap(confusion_matrix(y_test, y_predicted1), annot=True)

heatmap.set_title('confusion  matrix for testing', fontdict={'fontsize':12}, pad=12);




# for training dataset

y_predicted2 = rf_random.best_estimator_.predict(X_train)
print('training accuracy: ', accuracy_score(y_train, y_predicted2))

#
print('confusion martrix: ')

plt.subplot(122)
heatmap = sns.heatmap(confusion_matrix(y_train, y_predicted2), annot=True)

heatmap.set_title('confusion  matrix for training', fontdict={'fontsize':12}, pad=12);